In [1]:
#Import Libraries and Packages

import os
import json

# keras imports
from keras.preprocessing.image import img_to_array, load_img
from keras.utils.data_utils import get_file
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing import image
from keras.models import Model


import h5py
import numpy as np
import pickle as pk

import logging
logging.getLogger('tensorflow').disabled = True

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
#Loading Models for various checks
first_check = VGG16(weights='imagenet')
second_check = pk.load(open("/Users/rishitdholakia/Desktop/Car Damage Detection project/Car Damage check/classifier.pickle", 'rb')) #damaged vs whole
third_check = pk.load(open("/Users/rishitdholakia/Desktop/Car Damage Detection project/FRS/classifier.pickle", 'rb')) # Front Rear Side
fourth_check = pk.load(open("/Users/rishitdholakia/Desktop/Car Damage Detection project/Severity/classifier.pickle", 'rb')) # Minor Moderate Severe

In [4]:
# Loading  valid categories for identifying cars using VGG16
with open('/Users/rishitdholakia/Desktop/Car Damage Detection project/cat_counter.pk', 'rb') as f:
    cat_counter = pk.load(f)

In [5]:
cat_counter

Counter({('n04285008', 'sports_car'): 28.853904586285353,
         ('n04037443', 'racer'): 7.548823233228177,
         ('n03100240', 'convertible'): 22.27065716101788,
         ('n02974003', 'car_wheel'): 5.018784584943205,
         ('n02814533', 'beach_wagon'): 21.272268172120675,
         ('n03930630', 'pickup'): 17.554288666695356,
         ('n02930766', 'cab'): 6.648977704113349,
         ('n03710193', 'mailbox'): 0.11574728786945343,
         ('n03425413', 'gas_pump'): 0.09956557303667068,
         ('n03902125', 'pay-phone'): 0.07892029732465744,
         ('n02791270', 'barbershop'): 0.06867709010839462,
         ('n04461696', 'tow_truck'): 4.083389501320198,
         ('n03977966', 'police_van'): 2.266874913824722,
         ('n03770679', 'minivan'): 35.39320989861153,
         ('n03670208', 'limousine'): 12.064467553049326,
         ('n03445924', 'golfcart'): 0.8001158833503723,
         ('n03459775', 'grille'): 4.1132087131845765,
         ('n03769881', 'minibus'): 3.650426773121

In [6]:
# shortlisting top 27 Categories that VGG16 stores for cars (Can be altered for less or more)
cat_list  = [k for k, v in cat_counter.most_common()[:27]]

In [7]:
cat_list

[('n03770679', 'minivan'),
 ('n04285008', 'sports_car'),
 ('n03100240', 'convertible'),
 ('n02814533', 'beach_wagon'),
 ('n03930630', 'pickup'),
 ('n03670208', 'limousine'),
 ('n04037443', 'racer'),
 ('n02930766', 'cab'),
 ('n03594945', 'jeep'),
 ('n02974003', 'car_wheel'),
 ('n03459775', 'grille'),
 ('n04461696', 'tow_truck'),
 ('n03769881', 'minibus'),
 ('n03796401', 'moving_van'),
 ('n03977966', 'police_van'),
 ('n04065272', 'recreational_vehicle'),
 ('n02701002', 'ambulance'),
 ('n04252225', 'snowplow'),
 ('n03777568', 'Model_T'),
 ('n03445924', 'golfcart'),
 ('n04467665', 'trailer_truck'),
 ('n03417042', 'garbage_truck'),
 ('n03478589', 'half_track'),
 ('n03384352', 'forklift'),
 ('n02704792', 'amphibian'),
 ('n02965783', 'car_mirror'),
 ('n03032252', 'cinema')]

First Check - Car or not

In [8]:
CLASS_INDEX = None

In [9]:
CLASS_INDEX_PATH = '/Users/rishitdholakia/Desktop/Car Damage Detection project/imagenet_class_index.json'

In [10]:
def get_predictions(preds,top=5):
    global CLASS_INDEX
    
    #Load the jason file
    CLASS_INDEX = json.load(open(CLASS_INDEX_PATH))
    
    #get the results
    results = []
    for pred in preds:
        top_indices = pred.argsort()[-top:][::-1]
        result = [tuple(CLASS_INDEX[str(i)]) + (pred[i],) for i in top_indices]
        result.sort(key=lambda x: x[2], reverse=True)
        results.append(result)
    return results

In [11]:
def prepare_img_224(img_path):
    img = load_img(img_path, target_size=(224, 224))
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x

In [12]:
def car_categories_check(img_224, model):
    print ("Validating that this is a picture of your car...")
    out = model.predict(prepare_img_224(img_224))
    top = get_predictions(out, top=5)
    for j in top[0]:
        if j[0:2] in cat_list:
            print ("Car Check Passed!!!")
            print ("\n")
            return True 
    return False 

In [13]:
#car_categories_check('/Users/rishitdholakia/Desktop/damaged car.jpeg',VGG16(weights='imagenet'))
#print(prepare_img_224('/Users/rishitdholakia/Desktop/damaged car.jpeg'))

Second Check - Damaged or not

In [14]:
def get_features(img_path):
    model = Model(first_check.input,first_check.get_layer('fc1').output)
    image_size = (224, 224)
    img = image.load_img(img_path, target_size=image_size)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    feature = model.predict(x)
    flat = feature.flatten()
    flat = np.expand_dims(flat, axis=0)
    return flat

In [15]:
def car_damage_check(classifier,features):
    print ("Validating that damage exists...")
    train_labels = ['Damaged', 'Not Damaged']
    preds = classifier.predict(features)
    prediction = train_labels[preds[0]]
    
    if train_labels[preds[0]] == 'Damaged':
        print ("Validation complete - proceeding to location and severity determination")
        print ("\n")
        return True 
    else:
        return False

Third Check - Location Assesment

In [16]:
def location_assessment(classifier,features):
    print ("Validating the damage area - Front, Rear or Side")
    train_labels = ['Front', 'Rear', 'Side']
    preds = classifier.predict(features)
    prediction = train_labels[preds[0]]
    print ("Your Car is damaged at - " + train_labels[preds[0]])
    print ("Location assesment complete")
    print("\n")

Fourth Check - Location Assesment

In [17]:
def severity_assessment(classifier,features):
    print ("Validating the Severity...")
    train_labels = ['Minor Damage', 'Moderate Damage', 'Severe Damage']
    preds = classifier.predict(features)
    prediction = train_labels[preds[0]]
    print ("Your Car damage impact is - " + train_labels[preds[0]])
    print ("Severity assesment complete")
    print ("\n")

Engine

In [18]:
def engine(img_path):
    while True:
        try:
            g1 = car_categories_check(img_path, first_check)
            if g1 is False:
                print ("Are you sure this is a picture of your car? Please submit another picture of the damage.")
                print ("Hint: Try zooming in/out, using a different angle or different lighting.")
                break
            features=get_features(img_path)
            g2 = car_damage_check(second_check,features)

            if g2 is False:
                print ("Are you sure that your car is damaged? Please submit another picture of the damage.")
                print ("Hint: Try zooming in/out, using a different angle or different lighting.")
                break

            location_assessment(third_check,features)
            severity_assessment(fourth_check,features)
            break
        except:
            print ("Image not accessible. Please try again.")
            break

In [19]:
img_path = '/Users/rishitdholakia/Desktop/Car Damage Detection project/data3a/validation/03-severe/22.jpg'

In [20]:
engine(img_path)

Validating that this is a picture of your car...
Car Check Passed!!!


Validating that damage exists...
Validation complete - proceeding to location and severity determination


Validating the damage area - Front, Rear or Side
Your Car is damaged at - Front
Location assesment complete


Validating the Severity...
Your Car damage impact is - Severe Damage
Severity assesment complete


